In [1]:
from gurobipy import *
import numpy

n = 10
N = range(n)

numpy.random.seed(3143)

a = numpy.random.normal(0,1,n)
b = numpy.random.normal(0,1,n)  

print("a =", a)
print("b =", b)

c = numpy.zeros((n,n))

for i in N:
    for j in N:
        c[i,j] = round(numpy.sqrt((a[i]-a[j])**2+(b[i]-b[j])**2))
        
print("c =", c)
        
d = numpy.random.randint(-20,20,n)
d = d-sum(d)/n

f = numpy.random.randint(100,500,(n,n))

M = sum(abs(d))


a = [-0.51778725  0.29867848 -1.2737401   0.34818477  1.39393986  0.02841332
  0.4416515   0.75382786 -0.20117084 -0.09253233]
b = [ 0.26201227 -1.04646594 -0.59170027  0.47682533 -2.08183022  0.72101746
  0.99870966  0.48980161 -1.69822215  1.6292166 ]
c = [[0. 2. 1. 1. 3. 1. 1. 1. 2. 1.]
 [2. 0. 2. 2. 2. 2. 2. 2. 1. 3.]
 [1. 2. 0. 2. 3. 2. 2. 2. 2. 3.]
 [1. 2. 2. 0. 3. 0. 1. 0. 2. 1.]
 [3. 2. 3. 3. 0. 3. 3. 3. 2. 4.]
 [1. 2. 2. 0. 3. 0. 0. 1. 2. 1.]
 [1. 2. 2. 1. 3. 0. 0. 1. 3. 1.]
 [1. 2. 2. 0. 3. 1. 1. 0. 2. 1.]
 [2. 1. 2. 2. 2. 2. 3. 2. 0. 3.]
 [1. 3. 3. 1. 4. 1. 1. 1. 3. 0.]]


In [2]:
ip = Model("fctp")

x = ip.addVars(N, N, vtype=GRB.BINARY, name="x")
y = ip.addVars(N, N, name="y")

ip.addConstrs((quicksum(y[i,j] for j in N if j != i) - quicksum(y[j,i] for j in N if j != i) == d[i] for i in N), "dem")
ip.addConstrs((y[i,j] <= M*x[i,j] for i in N for j in N if i != j), "abre")

ip.setObjective(quicksum((c[i,j]*y[i,j] + f[i,j]*x[i,j]) for i in N for j in N if i != j))

ip.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-30
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 100 rows, 200 columns and 360 nonzeros
Model fingerprint: 0xdddeb498
Variable types: 100 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+01]
Found heuristic solution: objective 2219.7000000
Presolve removed 0 rows and 20 columns
Presolve time: 0.00s
Presolved: 100 rows, 180 columns, 360 nonzeros
Variable types: 90 continuous, 90 integer (90 binary)

Root relaxation: objective 1.448831e+02, 51 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  144

In [ ]:
master = Model()
master.Params.OutputFlag = 0

x = master.addVars(N, N, vtype=GRB.BINARY, name="x")
theta = master.addVar(name="theta")

master.setObjective(quicksum(f[i,j]*x[i,j] for i in N for j in N if i != j) + theta)

sub = Model()
sub.Params.OutputFlag = 0
sub.Params.InfUnbdInfo = 1

y = sub.addVars(N, N, name="y")

r1 = sub.addConstrs((quicksum(y[i,j] for j in N if j != i) - quicksum(y[j,i] for j in N if j != i) == d[i] for i in N), "dem")
r2 = sub.addConstrs((y[i,j] <= 0 for i in N for j in N if i != j), "abre")
sub.setObjective(quicksum(c[i,j]*y[i,j] for i in N for j in N if i != j))

In [ ]:
it = 0
while True:
    it += 1
    master.optimize()
    print(it, "obj =", master.objVal)
    
    for i in N:
        for j in N:
            if i != j:
                if x[i,j].x > 0.5:
                    r2[i,j].rhs = M
                else:
                    r2[i,j].rhs = 0

    sub.optimize()
    
    if sub.Status == GRB.INFEASIBLE:
        print("Infeasible")
        master.addConstr(sum(-r1[i].farkasDual*d[i] for i in N) + sum(-r2[i,j].farkasDual*M*x[i,j] for i in N for j in N if i != j) <= 0)
    elif sub.Status == GRB.OPTIMAL:
        print("Optimal")
        if sub.objVal <= theta.X + 0.001:
            break
        master.addConstr(sum(r1[i].pi*d[i] for i in N) + sum(r2[i,j].pi*M*x[i,j] for i in N for j in N if i != j) <= theta)

In [ ]:
master = Model()
master.Params.LazyConstraints = 1

x = master.addVars(N, N, vtype=GRB.BINARY, name="x")
theta = master.addVar(name="theta")

master.setObjective(quicksum(f[i,j]*x[i,j] for i in N for j in N if i != j) + theta)

In [ ]:
def cutCB(m, where):
    if where == GRB.Callback.MIPSOL:
                    
        for i in N:
            for j in N:
                if i != j:
                    if m.cbGetSolution(x[i,j]) > 0.5:
                        r2[i,j].rhs = M
                    else:
                        r2[i,j].rhs = 0
    
        sub.optimize()
        
        if sub.Status == GRB.INFEASIBLE:
            print("Infeasible")
            m.cbLazy(sum(-r1[i].farkasDual*d[i] for i in N) + sum(-r2[i,j].farkasDual*M*x[i,j] for i in N for j in N if i != j) <= 0)
        elif sub.Status == GRB.OPTIMAL:
            print("Optimal")
            if sub.objVal > m.cbGetSolution(theta) + 0.001:
                m.cbLazy(sum(r1[i].pi*d[i] for i in N) + sum(r2[i,j].pi*M*x[i,j] for i in N for j in N if i != j) <= theta)

In [ ]:
master.optimize(cutCB)